# MGP and MI exchange volumes
The dataset *GME_MGP_MI_QUANTITA* refers to the total volume traded (both sell and buy) for each hours in the MGP and MI markets.

In [21]:
import sys
import os
import pandas as pd

# Add the source directory to the system path
sys.path.append(os.path.abspath('../../src'))
from open_data import fetch_db_table_sqlserver16
import utils
import importlib
# Reload the module
importlib.reload(utils)

# SQL query to fetch data
sql_query = "SELECT * FROM GME_MGP_MI_QUANTITA" 
volumes = fetch_db_table_sqlserver16(sql=sql_query, verbose=False)
volumes = volumes.sort_values(by=['FLOWDATE','FLOWHOUR'])

volumes

c:\imbalance_forecast\src\open_data.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, con, params=parameters)


,FLOWDATE,FLOWHOUR,MARKET,TOTAL_PURCHASES,NAT_PURCHASES,CNOR_PURCHASES,CSUD_PURCHASES,NORD_PURCHASES,SARD_PURCHASES,SICI_PURCHASES,...,ROSN_SALES,SLOV_SALES,SVIZ_SALES,BSP_SALES,MALT_SALES,XAUS_SALES,XFRA_SALES,TOTITABSP_SALES,TOTITABSP_PURCHASES,LOADTIME
24,20241004,1,MGP,26381.579,26381.579,2124.022,4534.366,14885.499,763.908,1547.272,...,None,0.0,2909.0,280.0,0.0,234.0,1700.0,23947.579,26381.579,2024-10-04 18:22:30.000
48,20241004,1,MI1,1218.268,1125.417,89.741,120.655,605.125,14.501,46.858,...,None,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,2024-10-04 18:22:35.400
72,20241004,1,MI2,502.341,629.282,18.932,129.258,240.581,21.865,55.090,...,None,0.0,3.7,0.0,0.0,0.0,0.0,NaN,NaN,2024-10-04 18:22:39.830
25,20241004,2,MGP,25258.886,25256.182,2035.433,4339.167,14229.233,731.432,1453.404,...,None,0.0,2898.5,280.0,0.0,234.0,1700.0,22824.886,25258.886,2024-10-04 18:22:30.003
49,20241004,2,MI1,1443.012,1400.190,74.013,116.721,958.660,11.576,42.725,...,None,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,2024-10-04 18:22:35.400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1169,20241018,20,MGP,38585.639,38599.639,3276.177,6726.614,21448.301,999.159,2225.382,...,None,0.0,2541.0,284.0,0.0,204.0,3679.0,34648.367,38585.639,2024-10-17 13:10:40.780
1170,20241018,21,MGP,36139.127,36140.409,3002.641,6393.776,20174.919,977.016,2251.633,...,None,0.0,2561.0,284.0,0.0,204.0,3679.0,31874.883,36139.127,2024-10-17 13:10:40.780
1171,20241018,22,MGP,32986.628,33000.856,2709.187,5772.425,18634.326,893.690,1993.313,...,None,0.0,2340.0,569.0,0.0,408.0,3391.0,28687.628,32986.628,2024-10-17 13:10:40.780
1172,20241018,23,MGP,30100.149,30108.049,2507.514,5249.867,16962.422,823.837,1812.719,...,None,0.0,2340.0,569.0,0.0,408.0,3391.0,25801.149,30100.149,2024-10-17 13:10:40.780


In [22]:
market_labels = volumes['MARKET'].unique()

print(market_labels)

['MGP' 'MI1' 'MI2' 'MI3']


In [23]:
# Convert FLOWDATE to a string and then to datetime (YYYYMMDD format)
volumes['FLOWDATE'] = pd.to_datetime(volumes['FLOWDATE'].astype(str), format='%Y%m%d')
# Subtracting one hour from FLOWHOUR
volumes['FLOWHOUR'] = volumes['FLOWHOUR'] - 1
# Convert FLOWHOUR to timedelta (number of hours) and add it to FLOWDATE
volumes['ORAINI'] = volumes['FLOWDATE'] + pd.to_timedelta(volumes['FLOWHOUR'], unit='h')
# Dropping the old FLOWDATE and FLOWHOUR columns
volumes = volumes.drop(columns=['FLOWDATE', 'FLOWHOUR'])
volumes.set_index('ORAINI', inplace=True)

In [24]:
mgp_volumes = volumes[volumes['MARKET'] == 'MGP']
mgp_volumes = mgp_volumes.drop(columns=['MARKET'])

mgp_volumes

,TOTAL_PURCHASES,NAT_PURCHASES,CNOR_PURCHASES,CSUD_PURCHASES,NORD_PURCHASES,SARD_PURCHASES,SICI_PURCHASES,SUD_PURCHASES,AUST_PURCHASES,BRNN_PURCHASES,...,ROSN_SALES,SLOV_SALES,SVIZ_SALES,BSP_SALES,MALT_SALES,XAUS_SALES,XFRA_SALES,TOTITABSP_SALES,TOTITABSP_PURCHASES,LOADTIME
ORAINI,,,,,,,,,,,,,,,,,,,,,
2024-10-04 00:00:00,26381.579,26381.579,2124.022,4534.366,14885.499,763.908,1547.272,1708.231,0.0,None,...,None,0.0,2909.0,280.0,0.0,234.0,1700.0,23947.579,26381.579,2024-10-04 18:22:30.000
2024-10-04 01:00:00,25258.886,25256.182,2035.433,4339.167,14229.233,731.432,1453.404,1641.941,0.0,None,...,None,0.0,2898.5,280.0,0.0,234.0,1700.0,22824.886,25258.886,2024-10-04 18:22:30.003
2024-10-04 02:00:00,24804.218,24779.340,1997.856,4247.864,14015.772,719.020,1414.144,1595.602,0.0,None,...,None,0.0,2877.0,280.0,0.0,234.0,1700.0,22370.218,24804.218,2024-10-04 18:22:30.003
2024-10-04 03:00:00,24457.711,24457.711,2031.358,4139.501,13743.992,716.087,1395.746,1571.391,0.0,None,...,None,0.0,3022.0,679.0,0.0,265.0,1651.0,22269.025,24457.711,2024-10-04 18:22:30.007
2024-10-04 04:00:00,24669.233,24669.233,2009.075,4166.908,13901.838,720.231,1448.271,1591.483,0.0,None,...,None,0.0,3080.0,679.0,0.0,265.0,1700.0,22292.381,24669.233,2024-10-04 18:22:30.007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-18 19:00:00,38585.639,38599.639,3276.177,6726.614,21448.301,999.159,2225.382,2390.855,0.0,None,...,None,0.0,2541.0,284.0,0.0,204.0,3679.0,34648.367,38585.639,2024-10-17 13:10:40.780
2024-10-18 20:00:00,36139.127,36140.409,3002.641,6393.776,20174.919,977.016,2251.633,2341.505,0.0,None,...,None,0.0,2561.0,284.0,0.0,204.0,3679.0,31874.883,36139.127,2024-10-17 13:10:40.780
2024-10-18 21:00:00,32986.628,33000.856,2709.187,5772.425,18634.326,893.690,1993.313,2102.930,0.0,None,...,None,0.0,2340.0,569.0,0.0,408.0,3391.0,28687.628,32986.628,2024-10-17 13:10:40.780
